In [3]:
import geopandas as gpd
import pandas as pd
import datetime
from cityiq import Config, CityIq
from cityiq.api import Asset, Location
from cityiq.scrape import EventScraper

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt


%load_ext autoreload
%autoreload 2

tz = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo
f = '/Users/eric/proj/virt-proj/data-project/sdrdl-data-projects/sandiego.gov/predix.io/prod-credentials.yaml'
config = Config(f)
ciq = CityIq(config, start_time=20190101T00:00:00)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
loc_rec = list(ciq.locations)

In [5]:
locations = gpd.GeoDataFrame(ciq.locations_dataframe, geometry='geometry')
assets = gpd.GeoDataFrame(ciq.asset_dataframe, geometry='geometry')

locations.head()

,locationUid,locationType,parentLocationUid,geometry
0,004361eb,WALKWAY,004361eb,LINESTRING (-117.1706148251287 32.726548905684...
1,00456472,TRAFFIC_LANE,00456472,LINESTRING (-117.1705245131968 32.726558517207...
2,0051796c,WALKWAY,0051796c,LINESTRING (-117.1666581391174 32.722214661862...
3,00537bf3,TRAFFIC_LANE,00537bf3,LINESTRING (-117.166581746989 32.7222178855665...
4,005f90ed,WALKWAY,005f90ed,LINESTRING (-117.1665182431566 32.723810889902...


In [55]:
assets.head()

,assetUid,assetType,parentAssetUid,mediaType,events,geometry
0,000223ee-a868-474b-abcb-12ff1bad00a3,CAMERA,131d67b7-daaf-4252-9202-5df347a6c2a3,"IMAGE,VIDEO",,POINT (-117.1600173 32.71143062)
1,0002e3bb-5a9a-4083-9a96-fad0d22877b9,MIC,e8beafbf-7836-48c3-bb14-440970eecd6d,AUDIO,,POINT (-117.1493049 32.70997207)
2,0003806e-e409-449f-aab1-fc95f6e88d4e,CAMERA,f91ec557-f03e-4189-8b83-5aa8d28fbf14,"IMAGE,VIDEO",,POINT (-117.1566029 32.70934863)
3,000a4ac7-224b-4fb7-a2e2-cd0592d2b2de,CAMERA,741ce488-72fa-4216-a65d-216f7bdf8b7d,"IMAGE,VIDEO",,POINT (-117.148666 32.71585673)
4,000b2365-5309-422a-9be6-1b7127ca18db,NODE,None,None,,POINT (-117.264282290807 32.8105285177251)


In [12]:
parking = [ e for e in loc_ref if e.locationType == 'PARKING_ZONE']

In [20]:
import metapack as mp
pkg = mp.open_package('http://library.metatab.org/sandiegodata.org-geography-2018.csv')
pkg

In [21]:
tracts = pkg.resource('tract_boundaries').geoframe()
tract_comm =  pkg.resource('tract_communities').dataframe()

In [53]:
t = gpd.GeoDataFrame(
    [ (e.locationUid, e.geometry) for e in parking],
    columns = ['locationUid','geometry'],
    geometry='geometry')
t.crs = tracts.crs
t['geometry'] = t.geometry.centroid

t = gpd.sjoin(gdf, tracts)
parking_geo = t.merge(tract_comm, on='geoid').drop(columns=['geometry', 'index_right'])
parking_geo.head()

,locationUid,geoid,city_name,city_code,community_name,community_cpcode,community_code
0,03e23a6ace01fc14a86ab250ce65b682,14000US00000006974,San Diego,SD,Downtown,4,DOW
1,0ni87m8vfbfkjixihah5,14000US00000006974,San Diego,SD,Downtown,4,DOW
2,17389570219b9f86583906e1adbe1f4c,14000US00000006974,San Diego,SD,Downtown,4,DOW
3,389a8ee517722dab58216eab0f617856,14000US00000006974,San Diego,SD,Downtown,4,DOW
4,3d87wafg2r6jixhhy9t,14000US00000006974,San Diego,SD,Downtown,4,DOW


In [56]:
parking_geo.to_csv('parking_geo.csv')